# US Data Charting
This workbooks analyzes and plots the latest US States data from the [COVID Tracking project](https://covidtracking.com/).

In [ ]:
from datetime import datetime, timedelta, timezone
import dateutil.parser as du_parser
import pandas as pd
import eloader as el
import eplotter as ep

# load from the data loader helper
(df_regions_daily, df_regions_population) = el.load_opencovid19_data()
(df_us_daily, df_us_states_daily, _) = el.load_covidtracking_us_data()
(df_it_daily, _) = el.load_pcmdpc_it_data()

# TEMP: remove france as it's spiking (on 6/15)
df_regions_daily = el.filter_rows_remove_equals(df_regions_daily, 'CountryName', 'France', 'spiking inconsistently')

df_countries_daily = el.fuse_daily_sources(df_regions_daily, df_us_daily, df_it_daily)

# add differentials to the dataset, and cleanup
el.add_canonical_differentials(df_countries_daily)
df_countries_daily = el.cleanup_canonical(df_countries_daily)
df_countries_daily = df_countries_daily.drop(columns=['dateChecked'])

# also add differentials to the US States set, and cleanup
el.add_canonical_differentials(df_us_states_daily, daily_series_col='RegionName')
df_us_states_daily = el.cleanup_canonical(df_us_states_daily)
df_us_states_daily = df_us_states_daily.drop(columns=['dateChecked'])

## US Aggregate

Confirmed cases in the US. Other countries shown as references.

In [ ]:
# plot, ranked by Confirmed cases
ranked_countries_by_confirmed_cases = ep.rank_data_by_metric(df_countries_daily, metric='Confirmed', unique_key='CountryCode')
#ranked_countries_by_population = ep.rank_data_by_metric(df_countries_daily, metric='Population', unique_key='CountryCode', max_results=10)
highlight_countries = ['United States of America', 'China', 'Italy', 'Spain', 'France', 'United Kingdom', 'South Korea', 'Japan', 'Brazil', 'India', 'Mexico', 'Nigeria', 'Russia', 'Chile']
ep.scatter_plot_by_series(
    df_countries_daily,
    x_key='X', y_key='Confirmed',
    series_key='CountryName', series_names=ranked_countries_by_confirmed_cases['CountryName'],
    y_log=True,
    # series_is_secondary=lambda df: df['CountryName'].any() not in list(ranked_countries_by_population['CountryName']),
    series_is_secondary=lambda df: df['CountryName'].any() not in highlight_countries or df['Confirmed'].iloc[-1] < 10000,
    series_secondary_width=1,
    # bounds=[64, el.date_to_day_of_year(datetime.now()) - 64 + 7, 100, None],
    bounds=[64, None, 1000, None],
    data_labels="legend", data_labels_align='right',
    line_style_non_first_series='dotted',
    title='US: Confirmed Cases',
    label_x="Day of 2020",
    stamp_1='Highlighted: highly populated or reference countries'
)

Deaths count in the US versus the rest of the world. Significant comparison points highlighted.

In [ ]:
ep.scatter_plot_by_series(
    df_countries_daily,
    x_key='X', y_key='Deaths',
    series_key='CountryName', series_names=ranked_countries_by_confirmed_cases['CountryName'],
    y_log=True,
    series_is_secondary=lambda df: df['CountryName'].any() not in highlight_countries or df['Confirmed'].iloc[-1] < 10000,
    series_secondary_width=1,
    bounds=[64, None, 100, None],
    data_labels="legend", data_labels_align='right',
    line_style_non_first_series='dotted',
    title='US: confirmed Deaths',
    label_x="Day of 2020",
    stamp_1='Highlighted: highly populated or reference countries'
)

### Table for US Aggregate data
Last 20 days of aggregated data.

In [ ]:
df_countries_daily[df_countries_daily['CountryCode'] == 'US'][-20:].drop(columns=['CountryCode', 'CountryName'])

## US States Charts

### Normalized by State Population

The following charts show the infection size normalized to the population of the state, to treat states equally and understand 'how many people per million' have been infected and eventually died. Secondarily, curves are shifted to the left to make everyone start from the starting point and be able to focus on trends independently on delays on the infection on states.

In [ ]:
def join_with_population(df, regional_df):
    df_reg = regional_df[['CountryCode', 'RegionCode', 'Population']]
    return df.join(df_reg.set_index(['CountryCode', 'RegionCode']), on=['CountryCode', 'RegionCode'], lsuffix='left')


def add_cases_per_million(df):
    df['ConfirmedPerMillion'] = df['Confirmed'] / (df['Population'] / 1E+06)
    df['DeathsPerMillion'] = df['Deaths'] / (df['Population'] / 1E+06)


def ranked_scatter_plot(df, metric_col, series_col, is_selected=None, is_secondary=None, intersection_at=None,
                        title=None, label_x=None, stamp_1=None):
    # select the series items
    df_selected = is_selected(df) if is_selected else df[df['Confirmed'] > 10000]
    df_by_metric = ep.rank_data_by_metric(df_selected, metric=metric_col, unique_key=series_col)
    y_max = int(round(df_selected[metric_col].max() * 1.1))

    # if intersection, label things differently
    if not title:
        title = metric_col + " by " + series_col
        if intersection_at: title = title + ', since crossing ' + str(intersection_at)
    if not label_x:
        label_x = 'Days since crossing ' + str(intersection_at)

    # plot
    ep.scatter_plot_by_series(
        df, x_key='X', y_key=metric_col,
        series_key=series_col, series_names=df_by_metric[series_col],
        series_is_secondary=is_secondary if is_secondary else (lambda df: df['Population'].iloc[-1] < 1E+06 or df['Confirmed'].iloc[-1] < 8000),
        series_secondary_width=1,
        shift_x_to_intersect_y=intersection_at,
        bounds=[None, el.date_to_day_of_year(datetime.now()) - 20, 0, y_max],
        data_labels="series", data_labels_align="center",
        title=title, label_x=label_x, stamp_1=stamp_1,
    )


# get the population, and continue where not zero
df_per_million = join_with_population(df_us_states_daily, df_regions_population)
df_per_million = df_per_million[df_per_million['Population'].notna()]
add_cases_per_million(df_per_million)

In [ ]:
# plot Confirmed per Million, after 2%
intersection_pct = 0.02
intersection_at = int(intersection_pct * 1E+06 / 100)
ranked_scatter_plot(df_per_million, 'ConfirmedPerMillion', 'RegionName', intersection_at=intersection_at,
                    title='Confirmed Cases per Million, by US State', stamp_1='starting since crossing ' + str(intersection_at),
                    label_x='Days since crossing "' + str(intersection_at) + ' per million"')

In [ ]:
# plot Deaths per Million, after 2%
intersection_pct = 0.001
intersection_at = int(intersection_pct * 1E+06 / 100)
ranked_scatter_plot(df_per_million, 'DeathsPerMillion', 'RegionName', intersection_at=intersection_at,
                    title='Deaths per Million, by US State', stamp_1='starting since crossing ' + str(intersection_at),
                    label_x='Days since crossing "' + str(intersection_at) + ' per million"')

This chart shows the mortality rate, defined as: Deaths / Total Positives. There are multiple factors to take into account on the numerator (in particular non-attributed deaths) and on the denominator (for example low-symptomatic cases, and non-tested cases) so the real values for the death rates are probably different.

For now this is a baseline estimation given the numbers we have.

In [ ]:
# states with the highest Confirmed
regions_by_cases = ep.rank_data_by_metric(df_us_states_daily, metric='Confirmed', unique_key='RegionName')
confirmed_top_count = regions_by_cases['Confirmed'].iloc[0].astype(int)
confirmed_sec_threshold = round(confirmed_top_count / 30).astype(int)
secondary_function = lambda df: df['Confirmed'].iloc[-1] < confirmed_sec_threshold

In [ ]:
# Mortality
regions_by_death_rate = ep.rank_data_by_metric(df_us_states_daily, metric='Death_rate', unique_key='RegionName')
ep.scatter_plot_by_series(
    df_us_states_daily,
    x_key='X', y_key='Death_rate',
    series_key='RegionName', series_names=regions_by_death_rate['RegionName'],
    series_is_secondary=secondary_function, series_secondary_width=1,
    y_filter='expo',
    bounds=[el.current_day_of_year() - 7*12, None, 0, 10],
    legend_decimals=1, legend_suffix='%',
    data_labels="legend", data_labels_align='right',
    title="Death rate by US State, in the last 8 weeks",
    label_x="Day of 2020", label_y="Reported deaths / Confirmed cases (percent)",
    stamp_1="Grayed-out: states with low case count yet"
)

### In Absolute Numbers

The following charts are for Confirmed cases. The first represents the total number of people declared 'Positive' (which can happen even after death). Note that the statistics do not include non-observable numbers such as deaths that are now tested for the virus or people that had low symptoms and did not get tested.

In [ ]:
# [plot] Days sice Case 5000, log
case_intersection = 5000
ep.scatter_plot_by_series(
    df_us_states_daily,
    x_key='X', y_key='Confirmed',
    series_key='RegionName', series_names=regions_by_cases['RegionName'],
    series_is_secondary=secondary_function,
    shift_x_to_intersect_y=case_intersection,
    y_log=True,
    bounds=[None, (el.current_day_of_year() - 66) * 1.5, None, None],
    data_labels="series", data_labels_align="center",
    title='Confirmed Cases since case ' + str(case_intersection) + ', by US State',
    label_x='Days since case ' + str(case_intersection),
    stamp_1="Grayed-out: low case count for now"
)

This chart tries to bring the curves together at case #500, to see the difference in regional behavior after hitting that infection size.

In [ ]:
# [plot] Day of the year, all series, higher than 100
weeks = 12
last_day = el.current_day_of_year()
first_day = last_day - 7*weeks
ep.scatter_plot_by_series(
    df_us_states_daily,
    x_key='X', y_key='Confirmed',
    series_key='RegionName', series_names=regions_by_cases['RegionName'],
    series_is_secondary=secondary_function,
    series_secondary_width=1,
    y_log=True, y_filter='expo',
    bounds=[first_day, last_day, 1000, None],
    data_labels="legend", data_labels_align='right',
    title='Confirmed cases by US State, last ' + str(weeks) + ' weeks',
    label_x="Day of 2020",
    stamp_1="Grayed-out: low case count"
)

This chart shows deaths by state by time.

In [ ]:
# [plot] Day of the year, all series, higher than 100
ep.scatter_plot_by_series(
    df_us_states_daily,
    x_key='X', y_key='Deaths',
    series_key='RegionName', series_names=regions_by_cases['RegionName'],
    series_is_secondary=secondary_function,
    series_secondary_width=1,
    y_log=True, y_filter='expo',
    bounds=[first_day, last_day, 50, None],
    data_labels="legend", data_labels_align='right',
    title='Deaths by US State, last ' + str(weeks) + ' weeks',
    label_x="Day of 2020",
    stamp_1="Grayed-out: low case count"
)

## Nationwide plots

In [ ]:
#df_countries_daily['dCases/Tests'] = df_countries_daily['dConfirmed'] / df_countries_daily['dTampons']

# redefining df_us_daily using the countries, as it's been cleaned up and daily canonicals have been added
df_us_daily = df_countries_daily[df_countries_daily['CountryCode'] == 'US']

#all_cols = ['Confirmed', 'Infectious', 'Deaths', 'Recovered', 'Hospitalized', 'Tampons', 'dConfirmed', 'dInfectious', 'dDeaths', 'dRecovered', 'dHospitalized', 'dTampons', 'Death_rate']
selected_cols = ['dConfirmed', 'dDeaths', 'dTampons', 'Hospitalized']
filter_name = 'none'
for col in selected_cols:
    col_name = 'Daily new ' + col[1:] if col.startswith('d') else col
    ep.scatter_plot_by_series(df_us_daily,
        x_key='X', y_key=col,
        y_filter=filter_name,
        series_key='CountryName',
        # legend_off=True,
        title="US - " + col_name,
        bounds=[50, None, None, None],
        label_x="Day of 2020", label_y=col_name,
        stamp_1="Since March 1, 2020",
    )

## Region plots

In [ ]:
# rank states by daily cases
regions_by_daily_cases = ep.rank_data_by_metric(df_us_states_daily, metric='dConfirmed', unique_key='RegionName', unique_pick='last')

# add a cases/test metric
df_us_states_daily['dCases/Tests'] = df_us_states_daily['dConfirmed'] / df_us_states_daily['dTampons']

#region_codes = ['FL']
region_codes = ['CA', 'AZ', 'TX', 'FL', 'GA', 'AL']
df_regions = []
for region_code in region_codes:
    df_region = df_us_states_daily[df_us_states_daily['RegionCode'] == region_code]
    df_regions.append(df_region)
df_us_states_subset = pd.concat(df_regions,  ignore_index=True)

#all_cols = ['dCases/Tests', 'Confirmed', 'Infectious', 'Deaths', 'Recovered', 'Hospitalized', 'Tampons', 'dConfirmed', 'dInfectious', 'dDeaths', 'dRecovered', 'dHospitalized', 'dTampons', 'Death_rate']
selected_cols = ['dConfirmed', 'dDeaths', 'dTampons', 'Hospitalized'] # 'dCases/Tests'
filter_name = 'none'
for col in selected_cols:
    col_name = 'Daily new ' + col[1:] if col.startswith('d') else col
    ep.scatter_plot_by_series(df_us_states_subset,
        x_key='X', y_key=col,
        y_filter=filter_name,
        series_key='RegionName', series_names=regions_by_daily_cases['RegionName'],
        title="US States - " + col_name,
        bounds=[115, None, 0, None],
        label_x="Day of 2020", label_y=col_name,
        stamp_1="Since March 1, 2020",
    )

## Tables for US Regions
States ranked by higher Confirmed cases.

In [ ]:
regions_by_cases.drop(columns=['CountryCode', 'CountryName'])

All regions ranked by higher Mortality rates.

NOTE: when confirmed cases are low ('Confirmed') in the table below, the 'Death_rate' is not significant, so it's up to you to filter and interpret the data below.

In [ ]:
regions_by_death_rate.drop(columns=['CountryCode', 'CountryName'])